In [1]:
#syooress CPU warn
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
import matplotlib.pyplot as plt

In [3]:
from numpy import genfromtxt
import numpy as np
import pandas as pd
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y = pd.read_csv('../Data/WISDM_y.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y)

/Users/sandeep/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
def create_series(x,y,timestep,overlap):

    slide_step = int(timestep*(1-overlap))
    data_num = int((len(x)/slide_step)-1)
  
    dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
    labels = list()

    for i in range(data_num):
        labels.append(y[slide_step*(i+1)-1])
        for j in range(timestep):
            dataset[i,j,:] = x[slide_step*i+j,:]

    return dataset,np.array(labels)

In [6]:
# y_pred.append("5")
#arrange 16 lines data into a series
#try to import time series in this way
import numpy as np
dataset,labels = create_series(x,y,16,0.5)
dataset.shape

(137275, 16, 3)

In [7]:
dataset = np.reshape(dataset,(len(dataset),len(dataset[0])*len(dataset[0,0]),1))
dataset.shape

(137275, 48, 1)

In [8]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((96092, 48, 1), (41183, 48, 1), (96092,), (41183,))

In [9]:
def trans_encoder(inputs, head_size, n_heads, ff_dim, drop=0.0):
  # MultiHeadAttention
    x = K.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = K.layers.MultiHeadAttention(key_dim=head_size, num_heads=n_heads, dropout=drop)(x, x)
    x = K.layers.Dropout(drop)(x)
    res = x + inputs

    x = K.layers.LayerNormalization(epsilon=1e-6)(res)
    x = K.layers.Conv1D(filters=ff_dim, kernel_size=1,activation="relu")(x)
    x = K.layers.Dropout(drop)(x)
    x = K.layers.Conv1D(filters=inputs.shape[-1],kernel_size=1)(x)
    return x + res

In [10]:
def create_model(input_shape, head_size, n_heads, ff_dim,
                 n_trans_blocks, mlp_units, drop=0.0, mlp_drop=0.0):

    inpts = K.Input(shape=input_shape)
    x = inpts
    for _ in range(n_trans_blocks):
        x = trans_encoder(x, head_size, n_heads, ff_dim, drop)

    x = K.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = K.layers.Dense(dim, activation="relu")(x)
        x = K.layers.Dropout(mlp_drop)(x)
    
    oupts = K.layers.Dense(6, activation="softmax")(x) 
    return K.Model(inpts, oupts)


In [11]:
x_train.shape

(96092, 48, 1)

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K

# 0. get ready
print("\nBegin Transformer classification demo ")
np.random.seed(1)
tf.random.set_seed(1)

# 1. load training and test data
r = x_train.shape[0]
c = x_train.shape[1]
x_train = x_train.reshape((r, c, 1))  # (3601, 500, 1)
print(x_train.shape)

r = x_test.shape[0]
c = x_test.shape[1]
x_test = x_test.reshape((r, c, 1))  # (1320, 500, 1)
print(x_test.shape)



Begin Transformer classification demo 
(96092, 48, 1)
(41183, 48, 1)


In [43]:
print("\nCreating Transformer model ")
input_shape = x_train.shape[1:]  # (500,1)

model = create_model(input_shape, head_size=256, n_heads=4,
                     ff_dim=4, n_trans_blocks=4, mlp_units=[128],
                     drop=0.25, mlp_drop=0.4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=K.optimizers.Adam(learning_rate=1.0e-4),
              metrics=["sparse_categorical_accuracy"])
# Uncomment below if you want to use EarlyStopping
# c_backs = [K.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

print("\nStarting training ")
model.fit(x_train, y_train, validation_split=0.2,
          epochs=30, batch_size=36, shuffle=True)
print("Training complete")


Creating Transformer model 

Starting training 
Epoch 1/30
2136/2136 [==============================] - 283s 132ms/step - loss: 2.3497 - sparse_categorical_accuracy: 0.4972 - val_loss: 0.9084 - val_sparse_categorical_accuracy: 0.7277
Epoch 2/30
2136/2136 [==============================] - 289s 135ms/step - loss: 1.0704 - sparse_categorical_accuracy: 0.6660 - val_loss: 0.7390 - val_sparse_categorical_accuracy: 0.7691
Epoch 3/30
2136/2136 [==============================] - 292s 137ms/step - loss: 0.8562 - sparse_categorical_accuracy: 0.7243 - val_loss: 0.6545 - val_sparse_categorical_accuracy: 0.7838
Epoch 4/30
2136/2136 [==============================] - 288s 135ms/step - loss: 0.7504 - sparse_categorical_accuracy: 0.7508 - val_loss: 0.5991 - val_sparse_categorical_accuracy: 0.7887
Epoch 5/30
2136/2136 [==============================] - 288s 135ms/step - loss: 0.6884 - sparse_categorical_accuracy: 0.7632 - val_loss: 0.5666 - val_sparse_categorical_accuracy: 0.7943
Epoch 6/30
2136/2136 

In [44]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/Transformer'
model.save(model_dir).lo


In [45]:
print('hi')

In [13]:
import tensorflow as tf
from helper import compute_metrics_base, compute_metrics, measure_cpu_utilization_and_run
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score
from pathlib import Path

def compute_metrics_base(model, x_test, y_test, model_dir):
    """
    Compute the accuracy of the TensorFlow model.

    :param model: TensorFlow model.
    :param x_test: Test dataset features.
    :param y_test: Test dataset labels.
    :param model_dir: Directory path of the model.
    :return: None
    """
    
    # Get the model's predictions
    predictions = model.predict(x_test)

    # Convert predictions to label format
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Given the error, it seems y_test is already in label format.
    # So, no need to use np.argmax on it.
    true_labels = y_test
    
    model_size = sum(f.stat().st_size for f in Path(model_dir).rglob('*'))
    print(f'Size of the model: {model_size/1024:.2f} KB')

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Accuracy on the test set: {accuracy:.2%}')



In [15]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/Transformer'
model = tf.keras.models.load_model(model_dir)
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics_base, model, x_test, y_test, model_dir)

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


1287/1287 [==============================] - 57s 44ms/step
Size of the model: 1642.83 KB
Accuracy on the test set: 83.46%
Elapsed time: 57.10 seconds
Average CPU utilization: 31.85%


In [16]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(tflite_quant_model, X_test, y_test, input_type='float32'):
    """
    Compute the accuracy of the TFLite model.

    :param tflite_quant_model: Quantized TFLite model.
    :param X_test: Test dataset features.
    :param y_test: Test dataset labels.
    :param input_type: The type of the input data ('float32' or 'int8').
    :return: None
    """

    interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
    interpreter.allocate_tensors()
    input_tensor_index = interpreter.get_input_details()[0]['index']
    
    model_size = len(tflite_quant_model)
    print(f"Size of TFLite model: {model_size/1024:.2f} KB")

    predictions = []
    for i in range(len(X_test)):
        if input_type == 'float32':
            input_data = X_test[i:i + 1].astype(np.float32)
        elif input_type == 'int8':
            input_data = (X_test[i:i + 1] * 255).astype(np.int8)
        elif input_type == 'uint8':
            input_data = (X_test[i:i + 1] * 255).astype(np.uint8)
        else:
            raise ValueError(f"Unsupported input_type: {input_type}")

        interpreter.set_tensor(input_tensor_index, input_data)
        interpreter.invoke()

        output_details = interpreter.get_output_details()[0]
        output_data = interpreter.get_tensor(output_details['index'])
        predicted_label = np.argmax(output_data)
        predictions.append(predicted_label)

    predictions = np.array(predictions)
    # Given the context, y_test is already in label format.
    # So, no need to use np.argmax on it.
    true_labels = y_test
    accuracy = accuracy_score(true_labels, predictions)

    print(f'Accuracy on the test set: {accuracy:.2%}')


In [17]:
#Implemented Dynamic Range Quantization

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_model_optimization.sparsity import keras as sparsity

#Load saved model
model = tf.keras.models.load_model(model_dir)


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False

# Now, convert the model
transformer_dynamicQ_tflite_model = converter.convert()


# Save the quantized model to a file
with open('/Users/sandeep/Desktop/BUCourses/Project/saved_models/transformer_dynamicQ_model.tflite', 'wb') as f:
    f.write(transformer_dynamicQ_tflite_model)


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpedp37fc2/assets


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpedp37fc2/assets


In [18]:
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics, transformer_dynamicQ_tflite_model, x_test, y_test, input_type='float32'
                                                )

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


Size of TFLite model: 116.67 KB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy on the test set: 83.48%
Elapsed time: 51.69 seconds
Average CPU utilization: 22.25%


###### 